# CS105 Final Project
Group Members: August Bennett, Jalen Shen, Johnny To, Justin Ly
## Dataset
https://www.icpsr.umich.edu/web/NACJD/studies/27521

NOTE: Dataset is VERY Large (~3000) variables.
### Inital Features
These were the features selected to be possibly useful for us to make our classification model.
| Variable Name 	| Description                                       	|
|---------------	|---------------------------------------------------	|
| NODR30A       	| USUAL # OF DRINKS PER DAY PAST 30 DAYS            	|
| RK5ALWK       	| RISK HAVING 5 OR MORE DRINKS ONCE OR TWICE A WEEK 	|
| GRSKD4_5      	| GREAT RISK: HAVE 4-5 ALC DRINKS PER DAY           	|
| ALCPHCTD      	| CONTD TO DRINK ALC DESPITE PHYS PRBS              	|
| ALCEMCTD      	| CONTD TO DRINK ALC DESPITE EMOT PRBS              	|
| ALCFMCTD      	| CONTD TO DRINK ALC DESPITE PRBS W/ FAM/FRNDS      	|
| TXYRINAD      	| RES ALC/DRG REH TRMT FOR ALC, DRUGS, OR BOTH      	|
| TXILALEV      	| RECEIVED TREATMENT FOR DRG OR ALC USE IN LIFETIME 	|
| GENDER_R      	|  MALE (& FEMALE)                                  	|
| AGE_YRS       	| AGE IN YEARS RECODE                               	|
| ALCTRY        	| AGE WHEN FIRST DRANK ALCOHOLIC BEVERAGE           	|
| ALCREC        	| TIME SINCE LAST DRANK ALCOHOLIC BEVERAGE          	|
| ALCYRTOT      	| TOTAL # OF DAYS USED ALCOHOL IN PAST 12 MOS       	|
| IRALCFY       	| ALCOHOL FREQUENCY PAST YEAR - IMPUTATION REVISED  	|
| TOTDRINK      	| TOTAL # OF DAYS USED ALCOHOL IN PAST 12 MOS       	|
| TXEVER        	| EVER RECEIVED ALCOHOL OR DRUG TREATMENT           	|
| ALLARRST      	| ANY ARREST                                        	|
| ALCYDAYS      	| # OF DAYS USED ALCOHOL IN PAST YEAR               	|

After researching and using the user guide that was given to us via the study we revised our features. The main reason was some features that we selected were missing a large amount of responses. (some participants may have decided to skip the question). Below is the revised feature list.
| Variable Name 	| Description                                       	|
|---------------	|---------------------------------------------------	|
| NODR30A       	| USUAL # OF DRINKS PER DAY PAST 30 DAYS            	|
| ALCTRY        	| AGE WHEN FIRST DRANK ALCOHOLIC BEVERAGE           	|
| ALCREC        	| TIME SINCE LAST DRANK ALCOHOLIC BEVERAGE          	|
| IRALCFY       	| ALCOHOL FREQUENCY PAST YEAR - IMPUTATION REVISED  	|
| TOTDRINK      	| TOTAL # OF DAYS USED ALCOHOL IN PAST 12 MOS       	|
| ALCYDAYS      	| # OF DAYS USED ALCOHOL IN PAST YEAR               	|
| IRSEX          	| GENDER                                            	|
| AGE_YRS       	| AGE IN YEARS RECODE                               	|
| BOOKED      	    | EVER ARRESTED AND BOOKED FOR BREAKING THE LAW         |

These new variables were cleaned and made into a new file called cs105_dataset.csv

